In [ ]:
import pandas as _hex_pandas
import datetime as _hex_datetime
import json as _hex_json

In [ ]:
hex_scheduled = _hex_json.loads("false")

In [ ]:
hex_user_email = _hex_json.loads("\"example-user@example.com\"")

In [ ]:
hex_run_context = _hex_json.loads("\"logic\"")

In [ ]:
hex_timezone = _hex_json.loads("\"America/Mexico_City\"")

In [ ]:
hex_project_id = _hex_json.loads("\"44582f09-f401-4c5e-b7af-648aeb4202b7\"")

In [ ]:
hex_project_name = _hex_json.loads("\"5_euro2024_predictions\"")

In [ ]:
hex_status = _hex_json.loads("\"\"")

In [ ]:
hex_categories = _hex_json.loads("[]")

In [ ]:
hex_color_palette = _hex_json.loads("[\"#4C78A8\",\"#F58518\",\"#E45756\",\"#72B7B2\",\"#54A24B\",\"#EECA3B\",\"#B279A2\",\"#FF9DA6\",\"#9D755D\",\"#BAB0AC\"]")

# The final stretch - predicting the Euro 2024 Winner! ⚽

![image](https://i.gifer.com/9aZd.gif)

---

### 1. Show Fixture Data

**Objective:** Provide an overview of the fixture data for the Euro Cup 2024.

- **Initial Fixture Table**: Make a copy of the fixture table to preserve the original data.

### 2. Prep Data for Prediction

**Objective:** Prepare the data for predictive modeling.

- **Data Preprocessing**: Clean and transform the fixture data as necessary for prediction tasks.

### 3. Predict Results

**Objective:** Utilize predictive models to forecast match outcomes.

- **Prediction Process**: Employ suitable algorithms to predict match results.
- **Handling Shootouts**: Implement mechanisms to manage shootouts for knockout stage predictions.

### 4. Show Results

**Objective:** Present the predicted results of the Euro Cup 2024 matches.

- **Display Predicted Outcomes**: Showcase the forecasted results of each match.

In [ ]:
import snowflake.snowpark
from snowflake.snowpark.session import Session
from snowflake.snowpark import Window
from snowflake.snowpark import functions as F   
from snowflake.snowpark.functions import udf, udtf
from snowflake.snowpark.types import IntegerType, FloatType, StringType, StructField, StructType, DateType
from snowflake.ml.registry import Registry
    
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import hextoolkit
hex_snowflake_conn = hextoolkit.get_data_connection('SCS-SIMON-EURO2024')
session = hex_snowflake_conn.get_snowpark_session()

In [ ]:
# add version tracking
app_tag = {
    "origin": "sf_sit",
    "name": "hol_sport_predict",
    "version": '{major: 1, minor: 0}'
}

session.query_tag = app_tag

In [ ]:
user_name = session.sql('select current_user()').collect()[0][0]

In [ ]:
# let's load our model from the registry into memory

reg = Registry(session=session)

mv = reg.get_model("EURO_24_GAME_PREDICT").default

# Group Stage

In [ ]:
# adjustable threashold.
#
# setting at 0.5 introduces too many penalty shootouts

win_threshold = 0.45

# we can also use streamlit controller for this (DON'T DO THIS YET)
#win_threshold = st.slider(label='Threshold', min_value=0.0 ,max_value=1.0 ,value=0.45)

In [ ]:
# lets make a copy of the fixtures, one with original data and then one we'll update as we go along
df_fixture_copy = session.table('fixture')
df_fixture_copy.write.save_as_table(f'fixture_{user_name}',mode='overwrite')

# get list of fixtures for the groups stages
df_round_1 = (
    session.table(f'fixture_{user_name}')
    .select(
        F.col('"MATCH NUMBER"').alias("id"),
        F.col('"ROUND NUMBER"').alias('round'),
        F.to_date(F.col('"DATE"'), "DD/MM/YYYY HH24:MI").alias("date"),
        F.col('"HOME TEAM"').alias('team_1'),
        F.col('"AWAY TEAM"').alias('team_2'),
        F.col('"GROUP"').alias('group')
    )
)

df_round_1.show(36)

---------------------------------------------------------------------
|"ID"  |"ROUND"  |"DATE"      |"TEAM_1"     |"TEAM_2"     |"GROUP"  |
---------------------------------------------------------------------
|1     |1        |2024-06-14  |Germany      |Scotland     |Group A  |
|2     |1        |2024-06-15  |Hungary      |Switzerland  |Group A  |
|3     |1        |2024-06-15  |Spain        |Croatia      |Group B  |
|4     |1        |2024-06-15  |Italy        |Albania      |Group B  |
|5     |1        |2024-06-16  |Poland       |Netherlands  |Group D  |
|6     |1        |2024-06-16  |Slovenia     |Denmark      |Group C  |
|7     |1        |2024-06-16  |Serbia       |England      |Group C  |
|8     |1        |2024-06-17  |Romania      |Ukraine      |Group E  |
|9     |1        |2024-06-17  |Belgium      |Slovakia     |Group E  |
|10    |1        |2024-06-17  |Austria      |France       |Group D  |
|11    |1        |2024-06-18  |Türkiye      |Georgia      |Group F  |
|12    |1        |20

In [ ]:
# call the sproc that will run all our feature engineering code on the games we want to predict
#
# note - the 0 and 36 params refer to fixtures, we are only processing the feature engineering code for fixtures 0 through 36
# but...because we predict for both games twice we end up with 72 rows.

session.call('prep_prediction_data',0,36)

df_pred = session.table('data_for_predictions').order_by('id')
df_pred.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID"  |"TEAM_1"     |"TEAM_1_TTL_WINS"  |"TEAM_1_TTL_LOSSES"  |"TEAM_1_GOAL_DIFF"  |"TEAM_2_TTL_WINS"  |"TEAM_2_TTL_LOSSES"  |"TEAM_2_GOAL_DIFF"  |"TEAM_1_VS_TEAM_2_RANK"  |"NEUTRAL"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1     |Scotland     |3.0                |5.0                  |-6.0                |4.0                |5.0                  |-3.0                |23                       |1          |
|1     |Germany      |4.0                |5.0                  |-3.0                |3.0                |5.0                  |-6.0                |-23                      |0          |
|2     |Hungary      |6.0                |0.0                  |9

In [ ]:
# run predictions on all the group stage games

pred_df = session.table('data_for_predictions').order_by('id')
prediction = mv.run(pred_df, function_name="predict_proba")
prediction = prediction.with_column('output_game_outcome',F.iff(F.col('predict_proba_1') > win_threshold,1,0))
prediction.write.save_as_table('predictions',mode='overwrite',table_type='temp')

session.table('predictions').select(
    prediction.col('id'),
    prediction.col('team_1'),
    prediction.col('output_game_outcome')
).order_by('id').show()

----------------------------------------------
|"ID"  |"TEAM_1"     |"OUTPUT_GAME_OUTCOME"  |
----------------------------------------------
|1     |Germany      |1                      |
|1     |Scotland     |0                      |
|2     |Switzerland  |0                      |
|2     |Hungary      |0                      |
|3     |Spain        |0                      |
|3     |Croatia      |0                      |
|4     |Italy        |1                      |
|4     |Albania      |0                      |
|5     |Poland       |0                      |
|5     |Netherlands  |1                      |
----------------------------------------------



In [ ]:
# lets take the output of our predictions 
# 
# we restructure this into the groups sorted by points to determine the teams that go through
# 1st & 2nd, along with the 4 teams who ranked the highest in 3rd place (confusing right?) 

session.call('process_group_predictions')

# we should have our final tables for completed group stages

session.table('results_group_stage').filter(F.col('group') == 'Group A').order_by(F.col('group'),F.col('rank').asc()).show()
session.table('results_group_stage').filter(F.col('group') == 'Group B').order_by(F.col('group'),F.col('rank').asc()).show()
session.table('results_group_stage').filter(F.col('group') == 'Group C').order_by(F.col('group'),F.col('rank').asc()).show()
session.table('results_group_stage').filter(F.col('group') == 'Group D').order_by(F.col('group'),F.col('rank').asc()).show()
session.table('results_group_stage').filter(F.col('group') == 'Group E').order_by(F.col('group'),F.col('rank').asc()).show()
session.table('results_group_stage').filter(F.col('group') == 'Group F').order_by(F.col('group'),F.col('rank').asc()).show()

--------------------------------------------------------------------------
|"GROUP"  |"TEAM_NAME"  |"POINTS"  |"RANK"  |"QUALIFIED"  |"PLACE_CODE"  |
--------------------------------------------------------------------------
|Group A  |Germany      |9         |1       |YES          |1A            |
|Group A  |Hungary      |4         |2       |YES          |2A            |
|Group A  |Switzerland  |4         |3       |YES          |3A            |
|Group A  |Scotland     |0         |4       |             |4A            |
--------------------------------------------------------------------------

--------------------------------------------------------------------------
|"GROUP"  |"TEAM_NAME"  |"POINTS"  |"RANK"  |"QUALIFIED"  |"PLACE_CODE"  |
--------------------------------------------------------------------------
|Group B  |Italy        |5         |1       |YES          |1B            |
|Group B  |Spain        |5         |2       |YES          |2B            |
|Group B  |Croatia      

# Round of 16 

In [ ]:
# take the results of the group stage and insert the team names inmto the Round of 16 fixtures
# now we see who will be playing each each in the next phase

session.call('calculate_r16_games')
df_results = session.table(f'fixture_{user_name}').filter(F.col('"ROUND NUMBER"') == 'Round of 16').order_by('"MATCH NUMBER"')

# we can also use Streamlit to show snowpark dataframes (though this is only compatible for Snowflake Notebooks!)
# otherwise use df_results.show(8)

df_results.show()

-------------------------------------------------------------------------------------------------------------------------------
|"MATCH NUMBER"  |"ROUND NUMBER"  |"DATE"            |"LOCATION"             |"HOME TEAM"  |"AWAY TEAM"  |"GROUP"  |"RESULT"  |
-------------------------------------------------------------------------------------------------------------------------------
|37              |Round of 16     |29/06/2024 16:00  |Olympiastadion         |Hungary      |Spain        |NULL     |NULL      |
|38              |Round of 16     |29/06/2024 19:00  |BVB Stadion Dortmund   |Germany      |Denmark      |NULL     |NULL      |
|39              |Round of 16     |30/06/2024 16:00  |Arena AufSchalke       |England      |Türkiye      |NULL     |NULL      |
|40              |Round of 16     |30/06/2024 19:00  |Stadion Köln           |Italy        |Austria      |NULL     |NULL      |
|41              |Round of 16     |01/07/2024 16:00  |Düsseldorf Arena       |Netherlands  |Ukraine     

In [ ]:
# run the feature engineering code
# note the different fixture id parameters

session.call('prep_prediction_data',37,44)
session.table('data_for_predictions').order_by('id').show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID"  |"TEAM_1"     |"TEAM_1_TTL_WINS"  |"TEAM_1_TTL_LOSSES"  |"TEAM_1_GOAL_DIFF"  |"TEAM_2_TTL_WINS"  |"TEAM_2_TTL_LOSSES"  |"TEAM_2_GOAL_DIFF"  |"TEAM_1_VS_TEAM_2_RANK"  |"NEUTRAL"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|37    |Hungary      |6.0                |0.0                  |9.0                 |7.0                |1.0                  |19.0                |18                       |1          |
|37    |Spain        |7.0                |1.0                  |19.0                |6.0                |0.0                  |9.0                 |-18                      |1          |
|38    |Denmark      |7.0                |1.0                  |1

In [ ]:
# run predictions on all the games

pred_df = session.table('data_for_predictions').order_by('id')
prediction = mv.run(pred_df, function_name="predict_proba")
prediction = prediction.with_column('output_game_outcome',F.iff(F.col('predict_proba_1') > win_threshold,1,0))
prediction.write.save_as_table('predictions',mode='overwrite',table_type='temp')

session.table('predictions').select(
    prediction.col('id'),
    prediction.col('team_1'),
    prediction.col('output_game_outcome')
).order_by('id').show()

----------------------------------------------
|"ID"  |"TEAM_1"     |"OUTPUT_GAME_OUTCOME"  |
----------------------------------------------
|37    |Hungary      |0                      |
|37    |Spain        |1                      |
|38    |Denmark      |0                      |
|38    |Germany      |1                      |
|39    |England      |1                      |
|39    |Türkiye      |0                      |
|40    |Austria      |0                      |
|40    |Italy        |1                      |
|41    |Ukraine      |0                      |
|41    |Netherlands  |1                      |
----------------------------------------------



In [ ]:
# lets process the predictions into a format where we can see the winners
#
# note - since this is a knockout stage, any draws will have to be resolved by a penalty shootout

session.call('process_knockout_predictions')
session.table('results_ko_stage').order_by('match_id').show()

-----------------------------------------------------------------------------------------------------------------------------------
|"MATCH_ID"  |"ROUND"      |"DATE"            |"LOCATION"             |"TEAM_1"  |"TEAM_2"     |"WINNER"     |"PENALTY_SHOOTOUT"  |
-----------------------------------------------------------------------------------------------------------------------------------
|37          |Round of 16  |29/06/2024 16:00  |Olympiastadion         |Hungary   |Spain        |Spain        |                    |
|38          |Round of 16  |29/06/2024 19:00  |BVB Stadion Dortmund   |Denmark   |Germany      |Germany      |                    |
|39          |Round of 16  |30/06/2024 16:00  |Arena AufSchalke       |England   |Türkiye      |England      |                    |
|40          |Round of 16  |30/06/2024 19:00  |Stadion Köln           |Austria   |Italy        |Italy        |                    |
|41          |Round of 16  |01/07/2024 16:00  |Düsseldorf Arena       |Ukrai

# Quarter-finals

In [ ]:
# see the quarter final games as a result of the previous round

session.call('calc_knockout_games','Quarter Finals')
session.table(f'fixture_{user_name}').filter(F.col('"ROUND NUMBER"') == 'Quarter Finals').order_by('"MATCH NUMBER"').show()

--------------------------------------------------------------------------------------------------------------------------
|"MATCH NUMBER"  |"ROUND NUMBER"  |"DATE"            |"LOCATION"        |"HOME TEAM"  |"AWAY TEAM"  |"GROUP"  |"RESULT"  |
--------------------------------------------------------------------------------------------------------------------------
|45              |Quarter Finals  |05/07/2024 16:00  |Arena Stuttgart   |Spain        |Germany      |NULL     |NULL      |
|46              |Quarter Finals  |05/07/2024 19:00  |Volksparkstadion  |Portugal     |France       |NULL     |NULL      |
|47              |Quarter Finals  |06/07/2024 16:00  |Düsseldorf Arena  |Netherlands  |England      |NULL     |NULL      |
|48              |Quarter Finals  |06/07/2024 19:00  |Olympiastadion    |Italy        |Belgium      |NULL     |NULL      |
--------------------------------------------------------------------------------------------------------------------------



In [ ]:
# feature engineering code

session.call('prep_prediction_data',45,48)
session.table('data_for_predictions').order_by('id').show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID"  |"TEAM_1"     |"TEAM_1_TTL_WINS"  |"TEAM_1_TTL_LOSSES"  |"TEAM_1_GOAL_DIFF"  |"TEAM_2_TTL_WINS"  |"TEAM_2_TTL_LOSSES"  |"TEAM_2_GOAL_DIFF"  |"TEAM_1_VS_TEAM_2_RANK"  |"NEUTRAL"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|45    |Germany      |4.0                |5.0                  |-3.0                |7.0                |1.0                  |19.0                |8                        |0          |
|45    |Spain        |7.0                |1.0                  |19.0                |4.0                |5.0                  |-3.0                |-8                       |1          |
|46    |Portugal     |9.0                |1.0                  |2

In [ ]:
# run predictions on all the games

pred_df = session.table('data_for_predictions').order_by('id')
prediction = mv.run(pred_df, function_name="predict_proba")
prediction = prediction.with_column('output_game_outcome',F.iff(F.col('predict_proba_1') > win_threshold,1,0))
prediction.write.save_as_table('predictions',mode='overwrite',table_type='temp')

session.table('predictions').select(
    prediction.col('id'),
    prediction.col('team_1'),
    prediction.col('output_game_outcome')
).order_by('id').show()

----------------------------------------------
|"ID"  |"TEAM_1"     |"OUTPUT_GAME_OUTCOME"  |
----------------------------------------------
|45    |Germany      |0                      |
|45    |Spain        |1                      |
|46    |France       |1                      |
|46    |Portugal     |0                      |
|47    |Netherlands  |0                      |
|47    |England      |1                      |
|48    |Italy        |0                      |
|48    |Belgium      |1                      |
----------------------------------------------



In [ ]:
# process the predictions to see the winner 

session.call('process_knockout_predictions')
session.table('results_ko_stage').order_by('match_id').show()

------------------------------------------------------------------------------------------------------------------------------
|"MATCH_ID"  |"ROUND"         |"DATE"            |"LOCATION"        |"TEAM_1"  |"TEAM_2"     |"WINNER"  |"PENALTY_SHOOTOUT"  |
------------------------------------------------------------------------------------------------------------------------------
|45          |Quarter Finals  |05/07/2024 16:00  |Arena Stuttgart   |Germany   |Spain        |Spain     |                    |
|46          |Quarter Finals  |05/07/2024 19:00  |Volksparkstadion  |France    |Portugal     |France    |                    |
|47          |Quarter Finals  |06/07/2024 16:00  |Düsseldorf Arena  |England   |Netherlands  |England   |                    |
|48          |Quarter Finals  |06/07/2024 19:00  |Olympiastadion    |Belgium   |Italy        |Belgium   |                    |
---------------------------------------------------------------------------------------------------------------

# Semi-finals 

In [ ]:
# calculate the fixtures

session.call('calc_knockout_games','Semi Finals')
session.table(f'fixture_{user_name}').filter(F.col('"ROUND NUMBER"') == 'Semi Finals').order_by('"MATCH NUMBER"').show()

-------------------------------------------------------------------------------------------------------------------------------
|"MATCH NUMBER"  |"ROUND NUMBER"  |"DATE"            |"LOCATION"             |"HOME TEAM"  |"AWAY TEAM"  |"GROUP"  |"RESULT"  |
-------------------------------------------------------------------------------------------------------------------------------
|49              |Semi Finals     |09/07/2024 19:00  |Fußball Arena München  |Spain        |England      |NULL     |NULL      |
|50              |Semi Finals     |10/07/2024 19:00  |BVB Stadion Dortmund   |France       |Belgium      |NULL     |NULL      |
-------------------------------------------------------------------------------------------------------------------------------



In [ ]:
# feature engineering code

session.call('prep_prediction_data',49,50)
session.table('data_for_predictions').order_by('id').show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID"  |"TEAM_1"  |"TEAM_1_TTL_WINS"  |"TEAM_1_TTL_LOSSES"  |"TEAM_1_GOAL_DIFF"  |"TEAM_2_TTL_WINS"  |"TEAM_2_TTL_LOSSES"  |"TEAM_2_GOAL_DIFF"  |"TEAM_1_VS_TEAM_2_RANK"  |"NEUTRAL"  |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|49    |England   |6.0                |1.0                  |17.0                |7.0                |1.0                  |19.0                |-4                       |1          |
|49    |Spain     |7.0                |1.0                  |19.0                |6.0                |1.0                  |17.0                |4                        |1          |
|50    |Belgium   |6.0                |0.0                  |16.0               

In [ ]:
# run predictions on all the games

pred_df = session.table('data_for_predictions').order_by('id')
prediction = mv.run(pred_df, function_name="predict_proba")
prediction = prediction.with_column('output_game_outcome',F.iff(F.col('predict_proba_1') > win_threshold,1,0))
prediction.write.save_as_table('predictions',mode='overwrite',table_type='temp')

session.table('predictions').select(
    prediction.col('id'),
    prediction.col('team_1'),
    prediction.col('output_game_outcome')
).order_by('id').show()

-------------------------------------------
|"ID"  |"TEAM_1"  |"OUTPUT_GAME_OUTCOME"  |
-------------------------------------------
|49    |Spain     |0                      |
|49    |England   |1                      |
|50    |France    |0                      |
|50    |Belgium   |0                      |
-------------------------------------------



In [ ]:
# process the predictions to see the winner

session.call('process_knockout_predictions')
session.table('results_ko_stage').order_by('match_id').show()

-----------------------------------------------------------------------------------------------------------------------------
|"MATCH_ID"  |"ROUND"      |"DATE"            |"LOCATION"             |"TEAM_1"  |"TEAM_2"  |"WINNER"  |"PENALTY_SHOOTOUT"  |
-----------------------------------------------------------------------------------------------------------------------------
|49          |Semi Finals  |09/07/2024 19:00  |Fußball Arena München  |England   |Spain     |England   |                    |
|50          |Semi Finals  |10/07/2024 19:00  |BVB Stadion Dortmund   |France    |Belgium   |France    |YES                 |
-----------------------------------------------------------------------------------------------------------------------------



# Final 

In [ ]:
# calculate the final game

session.call('calc_knockout_games','Final')
session.table(f'fixture_{user_name}').filter(F.col('"ROUND NUMBER"') == 'Final').order_by('"MATCH NUMBER"').show()

------------------------------------------------------------------------------------------------------------------------
|"MATCH NUMBER"  |"ROUND NUMBER"  |"DATE"            |"LOCATION"      |"HOME TEAM"  |"AWAY TEAM"  |"GROUP"  |"RESULT"  |
------------------------------------------------------------------------------------------------------------------------
|51              |Final           |14/07/2024 19:00  |Olympiastadion  |France       |England      |NULL     |NULL      |
------------------------------------------------------------------------------------------------------------------------



In [ ]:
# feature engineering code

session.call('prep_prediction_data',51,51)
result_df = session.table('data_for_predictions').order_by('id').show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID"  |"TEAM_1"  |"TEAM_1_TTL_WINS"  |"TEAM_1_TTL_LOSSES"  |"TEAM_1_GOAL_DIFF"  |"TEAM_2_TTL_WINS"  |"TEAM_2_TTL_LOSSES"  |"TEAM_2_GOAL_DIFF"  |"TEAM_1_VS_TEAM_2_RANK"  |"NEUTRAL"  |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|51    |England   |6.0                |1.0                  |17.0                |7.0                |2.0                  |22.0                |2                        |1          |
|51    |France    |7.0                |2.0                  |22.0                |6.0                |1.0                  |17.0                |-2                       |1          |
--------------------------------------------------------------------------------

# And the winner is... 

In [ ]:
# run predictions on the final 

pred_df = session.table('data_for_predictions').order_by('id')
prediction = mv.run(pred_df, function_name="predict")
prediction.write.save_as_table('predictions',mode='overwrite',table_type='temp')

session.call('process_knockout_predictions')

df = session.table('results_ko_stage').order_by('match_id')
winner = df.collect()[0]['WINNER']

session.table('results_ko_stage').order_by('match_id').show()

------------------------------------------------------------------------------------------------------------------
|"MATCH_ID"  |"ROUND"  |"DATE"            |"LOCATION"      |"TEAM_1"  |"TEAM_2"  |"WINNER"  |"PENALTY_SHOOTOUT"  |
------------------------------------------------------------------------------------------------------------------
|51          |Final    |14/07/2024 19:00  |Olympiastadion  |England   |France    |England   |YES                 |
------------------------------------------------------------------------------------------------------------------



# Final Summary

Let's recap what we covered:

1) Ingested our raw data
2) Performed various steps of data transformation using Snowpark
3) Used **Snowpark ML** to run **Hyperparameter Tuning**, **Model Training** and stored our model in the **Snowflake Model Registry**
4) Registered various Python UDTFs and Stored Procedures for code reusability, to streamline our inference pipeline
5) Used our model to predict 51 matches and hopefuly saw England 🏴󠁧󠁢󠁥󠁮󠁧󠁿 or Italy 🇮🇹 taking the Euro 2024 trophy 🏆 home 

# What does Cortex say? ⚽

This isn't the right use of LLMs but for fun, let's see what [Cortex](https://docs.snowflake.com/user-guide/snowflake-cortex/llm-functions) has to say about the Euro 2024 outcome and compare to our prediction.

#### Note 
_The following models are only [supported](https://docs.snowflake.com/user-guide/snowflake-cortex/llm-functions#availability) in certain CSP regions at present so you will need to run this in a compatible setup._

In [ ]:
prompt = """
    Please respond to the following type of question with a single word along with the country flag emoji, naming a country. 
    The answer should be consider a classification response, for example:

    Question: Based on their performances up until 2022, which country was most likely to win the FIFA World Cup 2022?
    Response:England 🏴󠁧󠁢󠁥󠁮󠁧󠁿

    Question: Based on their performances up until 2020, which country was most likely to win the Euro 2020?
    Response:France 🇫🇷

    Now, based on historical performances and achievements in international and club football up to 2023,
    factoring in international players, and the location of the event, which country is most likely to win Euro 2024?
    
    Rules:
    1) Do not include any additional text or additional spaces before or after the country name. 
    2) Please strip out any whitespace before and after the answer.
"""

arctic = session.sql(f"select snowflake.cortex.complete(\'snowflake-arctic\','{prompt}')").collect()[0][0]
llama_3_8b = session.sql(f"select snowflake.cortex.complete(\'llama3-70b\','{prompt}')").collect()[0][0]
llama_3_70b = session.sql(f"select snowflake.cortex.complete(\'llama3-70b\','{prompt}')").collect()[0][0]
mistral_large = session.sql(f"select snowflake.cortex.complete(\'mistral-large\','{prompt}')").collect()[0][0]
mixtral_8_7b = session.sql(f"select snowflake.cortex.complete(\'mixtral-8x7b\','{prompt}')").collect()[0][0]
mixtral_8_7b = session.sql(f"select snowflake.cortex.complete(\'mixtral-8x7b\','{prompt}')").collect()[0][0]
reka_flash = session.sql(f"select snowflake.cortex.complete(\'reka-flash\','{prompt}')").collect()[0][0]

df = session.create_dataframe(
    [
        ('snowflake-arctic', arctic.lstrip().rstrip()),
        ('llama3-8b', llama_3_8b.lstrip().rstrip()), 
        ('llama3-70b', llama_3_70b.lstrip().rstrip()),
        ('mistral-large', mistral_large.lstrip().rstrip()), 
        ('mixtral-8x7b', mixtral_8_7b.lstrip().rstrip()),
        ('reka-flash', reka_flash.lstrip().rstrip()),
    ], 
    schema=["model", "prediction"])

df.show()

-----------------------------------
|"MODEL"           |"PREDICTION"  |
-----------------------------------
|snowflake-arctic  |Belgium 🇧🇪    |
|llama3-8b         |Germany 🇩🇪    |
|llama3-70b        |Germany 🇩🇪    |
|mistral-large     |France 🇫🇷     |
|mixtral-8x7b      |Germany 🇩🇪    |
|reka-flash        |Germany 🇩🇪    |
-----------------------------------

